# FVCOM unstructured mesh manipulation using Netcdf output and casename_obc.dat if any
#### Author: Jun Sasaki Coded on January 9, 2017, Updated on February 18, 2024
Netcdf outputから非構造格子を描画します．また，境界線を描き，さらに存在すれば開境界グリッド情報を読み込み，上書描画します．<br>
netcdfの読み込みはPyFVCOMとnetCDF4の両者を使っています．どちらか一方で十分ですが，目的に応じて使い分けています．<br>
tri.Triangulationのtriangles=nvのnvの3つのノードの値の順番がFVCOMでは時計回りとなっているようです．triの仕様では反時計回りのため，nvを反転する必要があります．これを行わないと，境界線の抽出が正しくできませんでした．なお，nvはFVCOMでは1始まりですので，1を引いて0始まりにします．

In [ ]:
from PyFVCOM.myread import ncread
#import mypyfvcom
import numpy as np
import matplotlib.pyplot as plt
# from PyFVCOM.read import ncread
import matplotlib.tri as tri
import netCDF4
import pandas as pd
fvcom_nc='../../goto2023/output/TokyoBay18_r16_crossed_0001_20s.nc'
fobc='../../goto2023/input/TokyoBay18_obc.dat' # = None if not exist
%matplotlib inline

## Taking time
The next cell consumes large time. Reconsider the way of reading netcdf.

In [ ]:
nc = netCDF4.Dataset(fvcom_nc, 'r').variables
FVCOM = ncread(fvcom_nc)
# Read open boundary node. delim_whitespace is separator for space and/or \t (tab)
if fobc is not None:
  df = pd.read_csv(fobc, header=None, skiprows=1, delim_whitespace=True)
# -1 because index in FVCOM starts from 1 while from 0 in Python
node_bc = df.iloc[:,1].values - 1

In [ ]:
node_bc # Open boundary nodes (index starts from 0.)

In [ ]:
print(nc.keys()) # Check variables of netcdf output

In [ ]:
# print(min(FVCOM['nv'][0])) # => 1 
nv = FVCOM['nv'].T - 1
x, y = FVCOM['x'], FVCOM['y']
xc, yc = FVCOM['xc'], FVCOM['yc']
triang = tri.Triangulation(x, y, triangles=nv)
# FVCOMでは時計回りに定義されているので，matplotlib.triの仕様である反時計回りに変更する
nv=nv[:,::-1]

In [ ]:
nv

In [ ]:
print(x.min(), x.max())
print(y.min(), y.max())

## Take a look at a figure and gets x and y ranges

In [ ]:
fig, ax = plt.subplots()
plt.gca().set_aspect('equal') ### Aspect ratio
ax.triplot(triang)

## Plot boundary lines and construct a fancy map figure
境界線を描きます．triag.neighbors[n,0], [n,1], [n,2]はcell nに接する3つのcell番号を返します．ただし，接するセルがない，すなわち境界を構成するcellの場合は-1を返します．よって，この-1の値をとるtriag.neighborsのインデックスが[n, j]であるとすると，そのcell nの境界を構成するedge（線分）はノードnv[n,j]とnv[n, (j+2%3]を結ぶものになります．何故nvのjに関わるindexがこのようになるかはよく分かりませんが，試行錯誤の結果，このようにするとうまくいくようです．<br>
この段階では開境界と固体境界の区別ができませんので，開境界情報を基に開境界を上書きする必要があります．

In [ ]:
nbe = np.array([[nv[n, j], nv[n, (j+2)%3]] for n in range(len(triang.neighbors)) for j in range(3) if triang.neighbors[n,j] == -1])

In [ ]:
import matplotlib.ticker as ticker
fig, ax = plt.subplots()
plt.gca().set_aspect('equal') ### Aspect ratio
plt.gca().patch.set_facecolor('0.8') ### Background color
# Plot triangular grids
ax.triplot(triang, color='b', linewidth=0.1)
# Plot boundary lines (solid and open)
for m in range(len(nbe)):
    ax.plot(x[nbe[m,:]], y[nbe[m,:]], color = 'r', linewidth = 0.5)
# Plot open boundary lines
ax.plot(x[node_bc[:]], y[node_bc[:]], color = 'b', linewidth = 1)
# Format x-axis and y-axis
xmin,xmax,ymin,ymax=375000, 420000, 3870000, 3952000
# xmin,xmax,ymin,ymax=-20000,25000,-50000,-20000 # Set range in m
#xmin,xmax,ymin,ymax=-20000,25000,-90000,-20000 # Set range in m
#xmin,xmax,ymin,ymax=-20000,25000,-90000,-20000 # Set range in m

ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10000))  # Set x major tick interval in m
ax.xaxis.set_minor_locator(ticker.MultipleLocator(2000))   # Set x minor tick interval in m
ax.yaxis.set_major_locator(ticker.MultipleLocator(10000))  # Set y major tick interval in m
ax.yaxis.set_minor_locator(ticker.MultipleLocator(2000))   # Set y minor tick interval in m
# Format in km without changing coordinates
def x_label(value, pos):  # m to km and start from 0
    global xmin
    value=int(value - xmin)//1000  # Integer
    return '{0:d}'.format(value)
def y_label(value, pos):  # m to km and start from 0
    global ymin
    value=int(value - ymin)//1000  # Integer
    return '{0:d}'.format(value)
ax.xaxis.set_major_formatter(ticker.FuncFormatter(x_label))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(y_label))
ax.set_xlabel('x (km)')
ax.set_ylabel('y (km)')
# Plot a marker at node number 10
ax.plot(x[10],y[10], marker = 'o', color='k', markersize = 5)
# Plot a marker at center of cell number 100
ax.plot(xc[100],yc[100], marker = 'o', color='y', markersize = 5)
# Put text using map coordinates
ax.text(400000,3875000, 'Tokyo Bay', fontsize=12)

In [ ]:
png='fvcom_grid2.png'
fig.savefig(png, dpi=1200, bbox_inches='tight')